In [38]:
import json
import os
from osmxtract import overpass, location
import geopandas as gpd
import requests
import json
import folium

segnalazioni_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=geojson"
confini_trento=".."+ os.sep + "data" + os.sep + "confini_trento.shp"

url_dettagli_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=marker&id=ID" #sostituire ID con l'ID della segnalazione

# https://sensorcivico.comune.trento.it/api/opendata/v1/help
# https://sensorcivico.comune.trento.it/api/opendata/v1/content/instantiatedClassList

In [39]:
confini = gpd.read_file(confini_trento,driver="ESRI Shapefile")

In [40]:
confini

,geometry
0,"POLYGON ((11.16341028998411 46.1484645421866, ..."


In [41]:
shapefile_usosuolo="http://webapps.comune.trento.it/cartografia/gis/dbexport?db=base&sc=uso_suolo&ly=usosuolo_view&fr=shp"
# aree_verdi_urbane = usosuolo[usosuolo.fumetto=='Aree verdi urbane'].plot()

In [42]:
bounds = (confini.bounds.miny[0],confini.bounds.minx[0], confini.bounds.maxy[0],confini.bounds.maxx[0])

In [43]:
query = overpass.ql_query(bounds,tag='leisure', values=['park'])

In [44]:
response = overpass.request(query)

In [45]:
leisure_park_geojson = overpass.as_geojson(response, 'polygon')

In [46]:
leisure_park_osm = gpd.GeoDataFrame.from_features(leisure_park_geojson)
leisure_park_osm[:5]

,access,addr:city,addr:housenumber,addr:postcode,addr:street,alt_name,amenity,barrier,ele,geometry,landuse,leisure,name,natural,old_name,source,surface,wikidata
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((11.0855931 46.0023505, 11.0855066 46...",playground,park,NaN,NaN,NaN,PCN ortofoto_colore 2006 - autorizzazione 1068...,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((11.089135 46.0058005, 11.0894869 46....",NaN,park,NaN,NaN,NaN,NaN,grass,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((11.1049814 46.0178392, 11.1049741 46...",NaN,park,Parco Romagnano,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((11.046005 46.0964925, 11.0457219 46....",NaN,park,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((11.0726028 46.0506752, 11.0725182 46...",NaN,park,Giardino di Vaneze,NaN,NaN,NaN,NaN,NaN


In [47]:
r = requests.get(segnalazioni_sensorcivico)

In [48]:
data = r.json()

In [49]:
data['features'][:2]

[{'type': 'Feature',
  'id': 17404,
  'properties': {'type': 'segnalazione',
   'name': 'Siepi che occupano il marciapiede',
   'popupContent': '<em>Loading...</em>'},
  'geometry': {'type': 'Point', 'coordinates': ['11.139826', '46.070819']}},
 {'type': 'Feature',
  'id': 17402,
  'properties': {'type': 'segnalazione',
   'name': 'Tombino rumoroso',
   'popupContent': '<em>Loading...</em>'},
  'geometry': {'type': 'Point', 'coordinates': ['11.129634', '46.036428']}}]

In [50]:
data.keys()

dict_keys(['type', 'features'])

In [51]:
for idx in range(len(data['features'])):
    data['features'][idx]['properties']['id'] = data['features'][idx]['id']

In [52]:
for k in range(len(data['features'])):
    data['features'][k]['geometry']['coordinates'][0] = float(data['features'][k]['geometry']['coordinates'][0])
    data['features'][k]['geometry']['coordinates'][1] = float(data['features'][k]['geometry']['coordinates'][1])


In [53]:
poi_sensorcivico = gpd.GeoDataFrame.from_features(data)

In [54]:
park_index = []
poi_index = []
for idxpoi, poi in poi_sensorcivico.iterrows():
    for idxpark, park in leisure_park_osm.iterrows():
        if (poi.geometry.within(park.geometry)):
            park_index.append(idxpark)
            poi_index.append(idxpoi)
            break

In [59]:
def getDesc(id):
    r = requests.get(url_dettagli_sensorcivico.replace("ID",str(id)))
    response = r.json()
    text = "" 
    if len(response) > 0:
        text = r.json()
    return text['content']

In [60]:
poi_sensorcivico_parchi = (poi_sensorcivico[poi_sensorcivico.index.isin(poi_index)]).copy()

In [61]:
poi_sensorcivico_parchi['desc'] = poi_sensorcivico_parchi['id'].apply(lambda x: getDesc(x))

In [62]:
poi_sensorcivico_parchi[:5]

,geometry,id,name,popupContent,type,desc
7,POINT (11.133946 46.088621),17374,Problema riccorrente zanzare,<em>Loading...</em>,segnalazione,"<div class=""clearfix"">\n<h4><a href=""/sensor/p..."
26,POINT (11.128603 46.068327),17236,Piastra di piazza Venezia lasciata a se stessa...,<em>Loading...</em>,reclamo,"<div class=""clearfix"">\n<h4><a href=""/sensor/p..."
27,POINT (11.06643 46.087683),17233,Segnalazione su Shelly App,<em>Loading...</em>,segnalazione,"<div class=""clearfix"">\n<h4><a href=""/sensor/p..."
74,POINT (11.12855 46.068281),16995,Allestimento e rifacimento del campetto da Bas...,<em>Loading...</em>,segnalazione,"<div class=""clearfix"">\n<h4><a href=""/sensor/p..."
115,POINT (11.11605 46.103043),16836,Sostegni altalena bambini rotti,<em>Loading...</em>,segnalazione,"<div class=""clearfix"">\n<h4><a href=""/sensor/p..."


In [63]:
poi_sensorcivico_parchi.to_file('poi_su_parchi.geojson', driver="GeoJSON")

In [70]:
map_trento = folium.Map([46.07,11.1192], zoom_start=13, tiles="OpenStreetMap")
map_trento.add_child(folium.GeoJson(poi_sensorcivico_parchi.to_json(),tooltip=folium.features.GeoJsonTooltip(fields=['desc'], labels=False, sticky=False,)))
# folium.GeoJson(poi_sensorcivico_parchi2.to_json()).add_to(map_trento)
map_trento